<a href="https://colab.research.google.com/github/hadisotudeh/product-color-classification/blob/main/ML%20model%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Product Color Classification

This project makes use of [FastAI](https://fast.ai) library to train a product color classifier on the [Digikala](https://digikala.com) product dataset released as part of their [data science](https://quera.ir/contest/assignments/20120/problems) competition.

In [ ]:
!git clone https://github.com/hadisotudeh/product-color-classification.git
!pip install fastai --upgrade

In [24]:
#hide
from fastai.vision.all import *
from PIL import Image

import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [27]:
train_path = Path("train")
test_path = Path("test")
split_index = 1

In [28]:
# run this cell if you are using google colab
repo_path = Path('product-color-classification')
train_path = Path.joinpath(repo_path, "train")
test_path = Path.joinpath(repo_path, "test")
split_index = 2

In [29]:
def get_color(file_path): return str(file_path).split("/")[split_index] 

dls = ImageDataLoaders.from_path_func(train_path, get_image_files(train_path), 
                                      label_func=get_color, seed=42, valid_pct=0.2,
                                      item_tfms=Resize(224))
learn = cnn_learner(dls, resnet101, metrics=accuracy)

learn.fine_tune(40, freeze_epochs=3)

epoch,train_loss,valid_loss,accuracy,time
0,1.875288,1.070994,0.709703,01:16
1,1.231248,0.958664,0.718524,01:19
2,0.921917,0.931325,0.746592,01:19


Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: ignored

In [ ]:
def show_image(x):
    """show the input image x
    Arguments:
        x {numpy array} -- input image x as a 3-dimensional numpay array
    Returns:
        None
    """

    plt.imshow(x)
    
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (img_height, img_width, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [ ]:
# prediction

decode_mapping = ['black', 'blue', 'brown', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'white', 'yellow']

encode_mapping = {'pink':1,
           'purple':2,
           'yellow':3,
           'orange':4,
           'white':5,
           'silver':6,
           'grey':7,
           'black':8,
           'red':9,
           'brown':10,
           'green':11,
           'blue':12,
}

results = []
for file in os.listdir(test_dir):
    dic = {}
    img = load(test_dir+file)
    y_prob = model.predict(img)
    y_classes = decode_mapping[y_prob.argmax(axis=-1)[0]]
    dic['file_name'] = file
    dic['color_id'] = encode_mapping[y_classes]
    results.append(dic)

In [ ]:
pd.DataFrame(results).to_csv("results_q4.csv",index=False)